In [1]:
# from Environments.TradingEnv_ray import Env_ray

import numpy as np
from numpy import random
import matplotlib.pyplot as plt
from numpy import load, save
import gym
import ray
from ray.tune.registry import register_env
from ray.rllib.agents.ppo import PPOTrainer, DEFAULT_CONFIG
import pandas as pd
import gc

In [95]:
first_col = pd.read_csv('Preprocessing/predictions_{}.csv'.format('label_0.25'), names = ['label_0.25'])
predicted_data = pd.DataFrame(first_col)
coeffs = [0.25, 0.5, 1, 2]
types = ['label', 'reversed_label', 'peaks', 'valleys']
for label_type in types:
    for coeff in coeffs:
        name = '{}_{}'.format(label_type, coeff)
        predicted_data[name] = pd.read_csv('Preprocessing/predictions_{}.csv'.format(name), names = [name])

In [155]:
low = predicted_data.min().to_numpy()
high = predicted_data.max().to_numpy()
np.save('Data/predictions/low.npy', low)
np.save('Data/predictions/high.npy', high)

In [2]:
df = pd.read_csv('Data/ground_truth/ground_truth.csv')

In [4]:
df_i = pd.read_csv('Data/indicator_dataset/indicator_dataset.csv')

In [3]:
df.candles

,peaks_0.5,valleys_0.5,peaks_0.25,valleys_0.25,peaks_1,valleys_1,peaks_2,valleys_2,label_0.25,reversed_label_0.25,...,reversed_label_2,reg_labels_0.25,reg_labels_0.5,reg_labels_1,reg_labels_2,price,bar:open,bar:close,bar:high,bar:low
0,0,0,0,0,0,0,0,0,1,0,...,0,2.13,2.13,3.53,16.56,135.38,135.49,135.38,135.50,135.26
1,1,0,1,0,1,0,1,0,1,0,...,0,2.24,2.24,3.64,16.67,135.27,135.34,135.27,135.50,135.26
2,1,0,1,0,1,0,1,0,1,0,...,0,2.02,2.02,3.42,16.45,135.49,135.26,135.49,135.59,135.16
3,1,0,1,0,1,0,1,0,1,0,...,0,1.22,1.22,3.10,3.10,136.29,135.45,136.29,136.41,135.27
4,0,1,0,1,0,0,0,0,0,1,...,0,-0.64,-1.09,1.40,14.43,137.51,136.30,137.51,137.57,136.28


In [7]:
candle_set = df.iloc[:, -4:]

In [8]:
candle_set.head()

,bar:open,bar:close,bar:high,bar:low
0,135.49,135.38,135.50,135.26
1,135.34,135.27,135.50,135.26
2,135.26,135.49,135.59,135.16
3,135.45,136.29,136.41,135.27
4,136.30,137.51,137.57,136.28


In [13]:
import datetime as dt
candle_set['time'] = candle_set.index
candle_set['time'] = dt.datetime(2018,1,1) + pd.TimedeltaIndex(candle_set['time'], unit='m')

In [22]:
candle_set.drop('json', axis=1, inplace=True)

In [24]:
candle_set.rename({"bar:open":'open',
                  'bar:close':'close',
                  'bar:high':'high',
                  'bar:low':'low'}, axis=1, inplace=True)

In [26]:
def transform(x):
    dct = {'time': {'year': x['time'].year,
                    'month': x['time'].month,
                    'day': x['time'].day,
                    'hour': x['time'].hour,
                    'minute': x['time'].minute},
           'open': x['open'],
           'high': x['high'],
           'low': x['low'],
           'close': x['close']}
    return dct

In [27]:
candle_set['json'] = candle_set.apply(lambda x: transform(x), axis=1)

In [28]:
candle_set['json'].to_json('candle_data.json')